<a href="https://colab.research.google.com/github/Haebuk/AutoML/blob/main/wandb_sweeps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q wandb

     |████████████████████████████████| 1.8 MB 7.7 MB/s 
     |████████████████████████████████| 170 kB 46.2 MB/s 
     |████████████████████████████████| 133 kB 55.7 MB/s 
     |████████████████████████████████| 138 kB 57.3 MB/s 
     |████████████████████████████████| 97 kB 650 kB/s 
     |████████████████████████████████| 63 kB 2.1 MB/s 
     |████████████████████████████████| 62 kB 984 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.26.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [2]:
!wandb login

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [9]:
# %load train_lib.py

def train():
    import numpy as np
    import tensorflow as tf
    import wandb
    config_defaults = {
        'layer1_size': 128,
        'layer1_activation': 'relu',
        'dropout_rate': 0.2,
        'optimizer': 'adam',
        'learning_rate': 0.01,
    }
    wandb.init(project='sweep-practice',
               config=config_defaults,
               magic=True)

    config = wandb.config

    fashion_mnist = tf.keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
    class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                   'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

    train_images.shape
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(config.layer1_size, activation=config.layer1_activation),
        tf.keras.layers.Dropout(config.dropout_rate),
        tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    if config.optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=config.learning_rate)
    else:
        opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(train_images, train_labels, epochs=5,
                  validation_data=(test_images, test_labels))


In [10]:
sweep_config = {
    'method': 'grid',
    'parameters': {
        'layer1_size': {
            'values': [64, 96, 128] # grid search
        },
        'layer1_activation': {
            'values': ['relu', 'sigmoid']
        },
        'dropout_rate': {
            'values': [0.1, 0.2, 0.3, 0.4, 0.5]
        },
        'optimizer': {
            'values': ['rmsprop', 'adam']
        },
        'learning_rate': {
            'values': [0.1, 0.01, 0.001]
        }
    }
}

In [11]:
import wandb
sweep_id = wandb.sweep(sweep_config, project='sweep-practice')

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


epoch,0
loss,0.60785
accuracy,0.7789
val_loss,0.55186
val_accuracy,0.7977
_runtime,9
_timestamp,1628229124
_step,0
best_val_loss,0.55186
best_epoch,0


epoch,▁
loss,▁
accuracy,▁
val_loss,▁
val_accuracy,▁
_runtime,▁
_timestamp,▁
_step,▁


Create sweep with ID: 4t5ceyzj
Sweep URL: https://wandb.ai/kade/sweep-practice/sweeps/4t5ceyzj


In [12]:
wandb.agent(sweep_id, function=train)

wandb: Ctrl + C detected. Stopping sweep.


In [20]:
with open('train.py', 'w') as f:
    f.write("""
import numpy as np
import tensorflow as tf
import wandb
config_defaults = {
    'layer1_size': 128,
    'layer1_activation': 'relu',
    'dropout_rate': 0.2,
    'optimizer': 'adam',
    'learning_rate': 0.01,
}
wandb.init(project='sweep-practice',
            config=config_defaults,
            magic=True)

config = wandb.config

fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

train_images.shape
train_images = train_images / 255.0
test_images = test_images / 255.0

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(config.layer1_size, activation=config.layer1_activation),
    tf.keras.layers.Dropout(config.dropout_rate),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

if config.optimizer == 'rmsprop':
    opt = tf.keras.optimizers.RMSprop(learning_rate=config.learning_rate)
else:
    opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)

model.compile(optimizer=opt,
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5,
                validation_data=(test_images, test_labels))

    """)

In [21]:
!head train.py


import numpy as np
import tensorflow as tf
import wandb
config_defaults = {
    'layer1_size': 128,
    'layer1_activation': 'relu',
    'dropout_rate': 0.2,
    'optimizer': 'adam',
    'learning_rate': 0.01,


In [23]:
!wandb agent kade/sweep-practice/tk863uzw

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2021-08-06 22:31:12.999051: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2021-08-06 22:31:12.999101: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-08-06 22:31:13.641634: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamExecutor with strength 1 edge matrix:
2021-08-06 22:31:13.641694: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1264]      0 
2021-08-06 22:31:13.641709: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1277] 0:   N 
2021-08-06 22:31:13.641881: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-06 22:31:13.642477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1)